### Innledning

Seaborn gjør det enkelt å lage mange standard plot med utgangspunkt i dataframe med en high-level api. Bruker matplotlib under the hood og jeg kan få tilgang til ax-objektene for å gjøre endringer manuelt med matplotlib syntax. 

Den har flat namespace der alle funksjonene er tilgjengelig på øverste nivået, men koden er organisert i moduler ut i fra hva slags type plot det er (relational, distributions, categorical). Hver modul har en rekke axes-level plots som fungerer analogt til custom matplotlib plot jeg kunne laget selv. Disse bruker ax=plt.gca() under hood for å koble på state, men vi kan også spesifisere ax=ax explisitt. I tillegg har hver modul en figure-level plot som returnerer FacetGrid objekt. Spesifiserer hvilken axes-level funksjon som skal brukes på hver subplot med kind=.

I utgangspunktet bruker jeg figure level. Det har fin api og håndterer subplot grid på god måte. Nedsiden er at alle argumentene er listet i funksjonscallen, så må ha kjennskap til disse fra axes-level plots i modulen. Det har også litt annen måte å spesifisere figsize enn matplotlib: spesifisere hight=inches, aspect=ratio. 

Unntaket er dersom jeg vil lage en figur med ulike typer plot på ulike subplots. I såfall bør jeg sette opp grid i matplotlib og kjøre axes-level funksjon separat på hver ax. 

Figure-level plotsene fra modulene over gir en wrapper til FacetGrid objektet. Litt usikker på om det blir aktuelt å jobbe med dette direkte. Dessuten finnes det to andre objekt for grids som ikke bare duplikere samme type plot på ulike subplots i rektangulær grid: vi har også jointplot og pairplot som kombinerer bivariat og marginal sammenheng i samme figur for henholdsvis én kombinasjon av variabler og samtlige kombinasjoner. 

Merk at vi ofte må gjøre litt preprocessing på dataframe vi sender inn for å få akkurat den visualiseringen vi vil ha. Spesielt hvis det er betingede fordelinger... må da bruke groupby. Kan ikke få gjort alt gjennom seaborn api.

Merk også at seaborn er laget for å bruke dataframe. Hvis vi må gjøre transformasjoner/subsetting, så konstruer alltid dataframe med innholdet i stedet for å jobbe eksplisitt med numerisk array.

### Figure level plots

Har felles api

I tillegg til å skille observasjon med ulik verdi med fysisk plassering i plottet kan vi også kommunisere informasjon om verdi av andre variabler med tre såkalte semantics: hue, style og size. Kan bruke flere sematnics på sammen variabel. Den prøver å inferre et godt valg, men kan overkjøre valget manuelt.

```python
g = figure_plot(data=df,
                x='x', y='y', # bestemme geometrisk plassering 
                hue=, # farge, hvilken colormap avhenger om rent kategorisk eller ordered/numerisk
                style=, # ulik marker
                size=, # ulik størrelse, bruker normalisert i stedet for proposjonal
                col=, # eget facet med subset av observasjon med ulik verdi av col
                col_wrap=, # maks antall facets på rad
                row=, # tilsvarer col_wrap=1, kan subsette etter både verdi på col og row.
                height=, # høyde til hver facet
                aspect=, # default er 1 (kvadrat)
               )
```

### Relational

```python
relplot(x='x', y='y',
        kind= # scatter eller line
        )
```

```python
scatterplot(x='x',y='y',
            ... # kan bruke semantics til å identifisere subsets av data
           )
```

```python
lineplot(x='x', y='y', # sorterer x
        ..., # kan få flere linjer ved å velge semantics
        estimator='mean', # kan velge hvilket sammendragsmål av y for hver x hvis flere verdier
        ci=None, # som default estimerer den standardfeil til estimator
        err_style='band' # kan enten ta band eller bars.. kommer litt ann på om x er diskret eller kont?
        )
```

#### Regresjon

Vi kan bruke modellering av E[y|x] til å visualisere releasjon. Dette er spesielt nyttig dersom vi har masse data. lmplot gir interface til FacetGrid slik at vi kan plotte regplots på ulike subsets på ulike facets

```python
lmplot(x='x',y='y',
       ... # mye samme argument som regplot
       cols= # mye samme argument som andre FacetGrid wrappers
      )
```

```python
regplot(x='x',y='y',
        x_bins, # har litt alternativer til scatter vi kan bruke hvis x er diskret
        order=2, # har alternative for ikke-lineær cef. order=2 gir polynom, eller så kan vi bruke
        lower=True, # lokal vektet regresjon (merk at ikke estimerer ci)
        ci=None, # alternativt en int i [0,100]
        logistic=True, # y må være 1/0 eller True/False. Ikke tilstrekkelig med ordered category.. burde vært.
        n_boot # kan være greit å spesifisere lavere verdi dersom logistisk med mye data.
        )
```

### Distributions

Fordel med empirisk kumulative distribusjon at den ikke trenger noen smoothing parameter (binwidth i kde/hist). Nedsiden er at det ikke er like lett å se sentraltends og spredning til fordelingen.


```python
distplot(...,
         rug= # spesifisere om vi vil ha rug til å visualisere individuelle observasjon
        )
```

```python
histplot(x='x', y='y', # kan bruke til å visualisere to dimensjonal density, men bruker nok ikke i praksis
         stat=, # count er default, kan bruke density for å få samme skala som kde. tror ikke andre er relevant
         bins=, # antall, har litt andre måter å kontrollere også ...
         multiple=, # layer er default. Føler at dodge må ha diskret data. stack kan være relevant: fylle opp univariat 
         element=, # bars er default, men kan ta step for å gjøre tydelig at tetthetsestimator
         fill=, # kan fjerne fill hvis bruker step
         )
```

```python
kdeplot(x='x', y='y',
        ... # multiple, fill er samme som i histplot
        clip = (c0, c1), # påvirker bare hvor density blir plottet, ikke selve estimeringen.
        common_norm = False # Hvis jeg vil at areal under hver kurve er lik 1
        )
```

```python
ecdfplot(x='x',
         ... # tror ikke har noen relevante argument
        )
```

### Categorical

Vi kan ha lyst til å visualisere betinget fordeling av en kontinuerlig variabel for hvert nivå av den kategoriske variabelen. For dette har vi to fremgangsmåter:
1. Variasjoner av scatterplot for hvert nivå av kategorisk
2. Variasjoner av boxplot for hvert nivå av kategorisk

Andre ganger vil jeg vise egenskap til fordeling av y i hvert bin. Bruker pointplot. Skjønner ikke hva jeg skal med bar.

Andre ganger vil ha univariat fordeling av den kategoriske.. bruker countplot. Litt usikker på hvordan jeg normaliserer til andel.



```python
catplot(x='x', y='y',
        order = [...]) # rekkefølge av verdi til kategorisk x. 
```

```python
stripplot(x='x',y='y', # kan brukes i kombinasjon med boxplot for å plotte underliggende punkt
          orient = [..], # kan spesifisere rekkefølge nivå av kategorisk blir plassert
          jitter=, # flytte litt horsiontalt slik at ikke overlapper
          size=
          )
```

```python
swarmplot(x='x',y='y', # ligner på stripplot, men punkt overlapper ikke. 
          ... # lignende args. Merk at metoden skalerer veldig dårlig til mange observer
          )
```

```python
boxplot(x='x', y='y',
       orient= # spesifiser 'h' eller 'v'. Må koordinere med valg av x og y variabel
       )
```

```python
violinplot(x='x', y='y', # alternativ måte å visualisere kernel density estimat
          ...)
```

```python
pointplot(x='x', y='y', # estimat av egenskap til betinget fordeling av y (f.eks sentraltendens) for hver nivå av x
          estimator=, # kan spesifisere funksjon. default=np.mean()
          ci=, # spesifisere størrelse på konf.intervall. default=0.95. Bruker bootstrap til å estimere
          ... # har litt params knyttet til bootstrapping
          )
```

```python
barplot(x='x', y='y', # virker analog til pointplot, men viser hel fuckings bar opp til y verdi... 
       ...
       )
```

```python
countplot('x', # kan spesifiere y i stedet, men må da sette orient='h'
          ... # vise count i hver bin. kan tenke på som histogram for kategorisk/diskret fordeling
         )
```    

### jointplot

Interface til JointGrid objekt så gjør at vi kan visualisere bivariat fordeling og marginale fordelinger i samme figur.

```python
jointplot(x='x',y='y',
         kind=
         'scatter' # scatterplot + histogram
         'kde' # konturlinjer av bivariat kde + univariat kde
         'hist' # bivariat hist + univariat hist
         )
```

### pairplot

Interface til PairGrid objekt for visualisering av parvise bivariate fordelinger

```python
pairplot(vars=cols, # som default tar den alle numerisk kolonner i df
         x_vars, y_vars, # tror alternativt jeg kan spesifisere x og y.. ikke-kvadratisk
         diag_kind, # default er 'auto' (avhengig av hue), alt: 'hist' og 'kde'
```

### Grids

Har sett på mange wrapper-funksjoner som jobber med ulike grids internt, men kan kanskje ha lyst til å jobbe med de direkte.

Kan bruke til å lage lattice-plot; mange gjentagelser av samme plot for ulike subset av data. Kan tydeliggjøre avvik fra mønster på god måte. 

```python
g = sns.FacetGrid(col=, col_wrap=) # diskret/kategorisk variabel. Kan eventuelt kombineres med row

g.map(func, *args, **kwargs) # kan ikke bruke x='x' av en eller annen grunn..
```

Vet ikke om det er aktuelt å jobbe med JointGrid og PairGrid eksplisitt.

### Annet

#### Heatmap

Korrelasjon reduserer scatterplot mellom to variabler til et enkelt tall. Kan bruke heatmap til å visualisere ved å gi det farge. Heatmap er egentlig en mer generell måte å visualisere 2d histogram. Gir oss et raskt overblikk over sammenheng mellom numeriske data... spesielt interessert i marginal sammenheng mellom uavhengige variabler og min avhengige. Etter å ha sett på heatmap hadde det vært greit å lage alle de marginale scatterplottene, men vet ikke hva som er best practice for å konstruere disse... tror det blir matplotlib eller pandas.plot api.

Kan også brukes til å visualisere confusion matrix i klassifisering

```python
sns.heatmap(data, # 2d array, i praksis ofte df.corr()
            annot=True, # kan alternativT sende inn array med samme størrelse som data.corr() for å annotere
            fmt='.2g', # spesifere decimal i annot
            vmin=-1, # default så inferres limits fra verdier i data. Kan alternativ spesifisere range med vmin, vmax
            vmax=1,
            center=0, # default cmap av om vi spesifiserer center. Bruker i så fall diverging. 
            cmap='coolwarm' # navn på innebygd cmap eller konstruere eget cmap-objekt
           )

cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(df.corr(), cmap=cmap)
```

Litt usikker på hvilken heatmap jeg liker best. 

### Style

Seaborn gir et high-level interface til matplotlib parametre.

Style:
1. axes_style() , gir ut dict med aktive parametre
2. set_style(), [darkgrid, whitegrid, dark, white, ticks], kan eventuelt overkjøre noen parametre

Scale:
1. plotting_context()
2. set_context(), [paper, notebook, talk, poster], poeng at jeg må ha større font/ticks hvis figur skal skaleres....


```python
sns.despine(fig, # som default tar den topp og høyre og andre args er optional
            top=True, right=True, # kan spesifiseres eksplisitt hvis jeg vil fjerne andre
            trim=True # kan være aktuelt i noen plot.. hmm.
            )
            
```

Kan endre parametre midlertidig ved å kombinere med with ..

### palette

Kan bestemme palette globalt med sns.set_palette("...") som tar litt ulike strings. Det er variasjoner av default palette:
deep, muted, pastel, bright, dark, and colorblind.

Har i tillegg såkalte COLOR BREWER PALETTES

sequential, divergent, .. hvis ordinal inndeling så vil jeg gjerne at det skal indikere rangeringen. Hvis ren kategorisk data trenger det ikek være mønster i farge.

### Eksempler

Barplot med fraksjon for hver kategori

```python
df = sns.load_dataset('titanic')

g = (df.groupby('class')['survived'].
      value_counts(normalize=True). # merk at .value_counts er series method..
      mul(100).
      rename('percent').
      reset_index().
      pipe((sns.catplot,'data'), x=x,y='percent',hue=y,kind='bar')) # merk at 'data' er keyword som tar element fra gb

g.ax.set_ylim(0,100)

for p in g.ax.patches:
    txt = str(p.get_height().round(2)) + '%'
    txt_x = p.get_x() 
    txt_y = p.get_height()
    g.ax.text(txt_x,txt_y,txt)
```